In [1]:
from libraries.utility import Utility as mutil
from libraries.db_base import db_base
from libraries.db_ins_fake_data import fake_data_to_db
from libraries.custom_excel_output import custom_excel_output
import pandas as pd
import pandasql as psql
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import warnings
import time
import os
import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))



In [2]:
dbprod = db_base("products")

Database : products, Connection Good: True


In [3]:
prod_info = dbprod.run_query_with_single_df(query_key="get_product_info")
prod_info.head(3)

,id,product_name,product_type
0,1,Terri's Date With Destiny,Event
1,2,Veronica's Patriotic Duty,TV Episode
2,3,Mark's Monster Madness,Event


In [4]:
dbfin = db_base("finance",svr_type='postsql')

Database : finance, Connection Good: True


In [5]:
fin_account_activity = dbfin.run_query_with_single_df(query_key="get_account_activity")
fin_account_activity.head(3)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,390,1,421,$4.49,2023-03-09 14:51:20,Home Video Rentals,"Brown, Smith and Pearson",Houston,Distribution Income Revenue 312106,Revenue
1,2,135,1,654,$8.94,2023-01-06 21:51:06,Social Media Platforms,Reid Inc,Houston,Interest Income Revenue 341656,Revenue
2,3,413,2,302,$17.28,2023-12-15 19:54:22,Home Video,"Melendez, Lamb and Jacobson",Houston,Rental Income Revenue 752346,Revenue


In [6]:
dbcust = db_base("customers",svr_type='mysql')

Database : customers, Connection Good: True


In [7]:
cust_products = dbcust.run_query_with_single_df(query_key="get_customer_product_history1")
cust_products.head(3)

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,1,Rebecca,Garcia,108,2023-11-15 09:06:32,2023-11-23 17:46:09,2023-12-21 23:40:56
1,1,Rebecca,Garcia,390,2023-03-10 02:54:39,2023-04-07 14:51:20,2023-12-21 23:40:56
2,1,Rebecca,Garcia,390,2023-03-09 14:51:20,2023-03-10 02:54:39,2023-12-21 23:40:56


In [8]:
sql = '''

select * from cust_products where id = 3

'''
df = psql.sqldf(sql)
df.head()

,id,f_name,l_name,product_id,purchase_dt,expiration_dt,created_dt
0,3,Laura,Woodward,54,2023-11-05 05:04:02.000000,2023-11-07 02:39:31.000000,2023-12-21 23:40:56.000000
1,3,Laura,Woodward,485,2023-09-29 16:05:09.000000,2023-12-09 21:15:47.000000,2023-12-21 23:40:56.000000
2,3,Laura,Woodward,485,2023-09-28 21:15:47.000000,2023-09-29 16:05:09.000000,2023-12-21 23:40:56.000000
3,3,Laura,Woodward,125,2023-05-15 19:31:33.000000,2023-11-09 07:07:27.000000,2023-12-21 23:40:56.000000
4,3,Laura,Woodward,125,2023-05-14 07:07:27.000000,2023-05-15 19:31:33.000000,2023-12-21 23:40:56.000000


In [9]:
fin_account_activity.head(1)

,id,product_id,customer_id,account_id,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
0,1,390,1,421,$4.49,2023-03-09 14:51:20,Home Video Rentals,"Brown, Smith and Pearson",Houston,Distribution Income Revenue 312106,Revenue


In [10]:
display(md("### Join everthing together in pandassql"))
sql = '''
select
distinct
    cp.id as customer_id,
    cp.f_name,
    cp.l_name,
    p.product_name,
    p.product_type,
    fa.amt_usd,
    fa.post_date,
    fa.channel_desc,
    fa.partner_desc,
    fa.location_name,
    fa.account_name,
    fa.account_type
from cust_products as cp
join prod_info as p
on 
    p.id = cp.product_id
join fin_account_activity fa
on 
 fa.customer_id = cp.id
 and
 fa.product_id = cp.product_id
'''
df = psql.sqldf(sql)


### Join everthing together in pandassql

In [11]:
dbcust = custom_excel_output(current_database="customers",svr_type='mysql')

Database : customers, Connection Good: True


In [12]:
# Clean up the post_date column
df['post_date'] = pd.to_datetime(df['post_date'])
df['post_date'] = df['post_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

u = mutil()
u.mkdir(os.path.join(u.get_this_dir(),"project_data","tempdir"))

display(md("### Example of Joined Data (Shuffled) and Excel Download"))

file_name = os.path.join(dbcust.get_this_dir(),"project_data","tempdir","customer_account_activity.xlsx")

df["amt_usd"] = df["amt_usd"].replace('[\$,]', '', regex=True).astype(float)

l_dfs = []
l_dfs.append(df)
dbcust.write_excel_from_dfs(list_of_dfs=l_dfs,file_name=file_name,add_subtotal_on_top=True)
href_tag = dbcust.get_embedded_href_tag_from_image_file(file_name)
dbcust.nukefile(file_name)
display(HTML(href_tag))
shuffled_df = df.sample(frac=1)
display(shuffled_df.head(5).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])]))

### Example of Joined Data (Shuffled) and Excel Download

,customer_id,f_name,l_name,product_name,product_type,amt_usd,post_date,channel_desc,partner_desc,location_name,account_name,account_type
3830,708,Kevin,White,Jennifer's Monster Madness,Bundle,11.660000,2023-02-08 01:16:25,Outdoor Events,Patterson Inc,Katy,Distribution Income Revenue 255451,Revenue
9174,1694,Laura,Marshall,Matthew's Amazing Race,TV Series,22.780000,2023-07-03 04:22:13,Museums and Cultural Institutions,Krueger PLC,Chillicothe,Distribution Income Revenue 886532,Revenue
8524,1575,Juan,Gray,Adam's Great War,Subscription,29.740000,2023-03-27 16:04:28,Home Video Rentals,Ramirez and Sons,Folsom,Rental Income Revenue 762168,Revenue
1589,294,Kevin,Zhang,James's Crazy Comedy,Bundle,25.820000,2023-07-28 07:15:10,Video-on-Demand (VOD) Services,"Clarke, Wilson and Ball",San Diego,Music Income Revenue 461795,Revenue
7359,1361,Kevin,Copeland,Benjamin's Great War,TV Series,25.970000,2023-09-10 07:42:16,Sports Networks,Boyle Inc,High Point,Rental Income Revenue 387657,Revenue


In [13]:
display(md("### Visualization of Key Data"))

import pandas as pd
import altair as alt

# Assuming df is your DataFrame
data = df

# For the pie chart, visualize the revenue per product type
pie_data = data.groupby('product_type')['amt_usd'].sum().reset_index()

# Calculate total revenue by city
city_revenue = data.groupby('location_name')['amt_usd'].sum().reset_index()
total_revenue = city_revenue['amt_usd'].sum()
city_revenue['revenue_percentage'] = (city_revenue['amt_usd'] / total_revenue) * 100

# Get the transaction count for each city
city_transactions = data['location_name'].value_counts().reset_index()
city_transactions.columns = ['location_name', 'transactions']

# Merge the revenue percentage data with the transaction count data
top_cities = city_transactions.merge(city_revenue, on='location_name')

# Take the top 10 cities based on transaction count
top_cities = top_cities.head(10)

# Define a more subdued color scheme
subdued_colors = ['#6baed6', '#bdd7e7', '#fdbe85', '#fdae6b', '#a1d99b', '#e5f5e0', '#bcbddc', '#9e9ac8', '#fcc5c0', '#fc9272']

# Creating a pie chart with a legend to the right and the subdued color scheme, formatting numbers in USD
pie_chart = alt.Chart(pie_data).mark_arc().encode(
    theta=alt.Theta(field='amt_usd', type='quantitative'),
    color=alt.Color(field='product_type', type='nominal', legend=alt.Legend(title='Product Type', orient='right'), scale=alt.Scale(range=subdued_colors)),
    tooltip=[alt.Tooltip('product_type:N'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
).properties(title='Revenue per Product Type', width=400)

# Creating a donut chart for the top 10 cities with a legend to the right and the subdued color scheme
donut_chart = alt.Chart(top_cities).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field='transactions', type='quantitative'),
    color=alt.Color(field='location_name', type='nominal', legend=alt.Legend(title='Top Cities', orient='right'), scale=alt.Scale(range=subdued_colors)),
    tooltip=[alt.Tooltip('location_name:N'), 
             alt.Tooltip('transactions:Q', title='Transactions', format=','), 
             alt.Tooltip('revenue_percentage:Q', title='Revenue %', format='.2f'),
             alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
).properties(title='Top 10 Cities by Transaction Count, Revenue Percentage, and Dollar Amount', width=400)

# Combine the charts
combined_charts = alt.hconcat(
    pie_chart, donut_chart
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

# To display the charts in the notebook, if you're running Jupyter Notebook or JupyterLab
combined_charts


### Visualization of Key Data

alt.HConcatChart(...)

In [14]:
import altair as alt

# ... [Your code for data loading and processing]

data = df

# Assuming 'data' is your DataFrame after cleaning
data['post_date'] = pd.to_datetime(data['post_date'])
data['amt_usd'] = pd.to_numeric(data['amt_usd'])

# Group the data by month and calculate the sum of 'amt_usd' in each month
monthly_revenue = data.groupby(data['post_date'].dt.to_period("M")).sum()['amt_usd'].reset_index()

# Convert the Period object to a string for 'post_date' column
monthly_revenue['post_date'] = monthly_revenue['post_date'].dt.strftime('%Y-%m')


# Altair Visualizations
# 1. Monthly Revenue Over Time
line = alt.Chart(monthly_revenue).mark_line().encode(
    x='post_date:T',
    y='amt_usd:Q'
)

points = alt.Chart(monthly_revenue).mark_point(size=150, opacity=0).encode(
    x='post_date:T',
    y='amt_usd:Q',
    tooltip=[alt.Tooltip('post_date:T', title='Date'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
)

monthly_revenue_chart = line + points
monthly_revenue_chart = monthly_revenue_chart.properties(
    title='Monthly Revenue Over Time',
    width=300,
    height=150
)
# 2. Distribution of Transaction Amounts with Sine Wave
transaction_distribution_chart = alt.Chart(data).mark_bar(color='#bdd7e7').encode(
    alt.X("amt_usd:Q", bin=alt.Bin(maxbins=30), title='Transaction Amount (USD)'),
    y='count()',
    tooltip=[alt.Tooltip('count()', title='Number of Transactions'), alt.Tooltip('amt_usd:Q', title='Amount', format='$,.2f')]
).properties(
    title='Distribution of Transaction Amounts',
    width=300,
    height=150
)  # Combine the sine wave chart with the transaction distribution chart

# Group the data by 'post_date' and 'product_type' and calculate the sum of 'amt_usd' in each group
revenue_product_type = data.groupby(['post_date', 'product_type']).sum()['amt_usd'].reset_index()

# Convert the 'post_date' column to a string for better visualization
revenue_product_type['post_date'] = revenue_product_type['post_date'].dt.strftime('%Y-%m')


# 3. Revenue by Product Type
revenue_product_chart = alt.Chart(revenue_product_type).mark_bar(color='#bdd7e7').encode(
    x='amt_usd:Q',
    y=alt.Y('product_type:N', sort='-x'),
    tooltip=[alt.Tooltip('product_type:N', title='Product Type'), alt.Tooltip('amt_usd:Q', title='Revenue', format='$,.2f')]
).properties(
    title='Revenue by Product Type',
    width=300,
    height=150
)

# Combine charts into a single visualization
combined_chart = alt.hconcat(monthly_revenue_chart, transaction_distribution_chart, revenue_product_chart).resolve_scale(
    color='independent'
)

alt.data_transformers.enable('json')

# Display the chart
combined_chart


alt.HConcatChart(...)

In [15]:
import glob
# Define the pattern to match files (starting with 'altair-data' and ending with '.json')
pattern = 'altair-data*.json'

mu = mutil()

# Get a list of matching files in the directory
matching_files = glob.glob(os.path.join(mu.get_this_dir(), pattern))

# Prompt the user for confirmation
if matching_files:
    print("The following files will be deleted:")
    for file_path in matching_files:
        print(file_path)

    confirmation = input("Do you want to proceed? (y/n): ").strip().lower()

    if confirmation == 'y':
        # Loop through the matching files and delete them
        for file_path in matching_files:
            os.remove(file_path)
        print("Files have been deleted.")
    else:
        print("No files were deleted.")
else:
    print("No matching files found in the directory.")

The following files will be deleted:
/data/school/MIDS/sql_server_work/j_nbks/altair-data-5d30d4c1fa3de8492ecbd8356aeb400d.json
/data/school/MIDS/sql_server_work/j_nbks/altair-data-7ad9e7042081830147736308b248567e.json
/data/school/MIDS/sql_server_work/j_nbks/altair-data-8284a6cdf994c3a85950582d1edec09e.json
Do you want to proceed? (y/n): y
Files have been deleted.
